In [ ]:
import json
import numpy as np
import pandas as pd
import folium

# データ読み込み
df = pd.read_json("https://covid.ourworldindata.org/data/owid-covid-data.json")
df = df[df.columns.drop(list(df.filter(regex='OWID')))] 

In [ ]:
# 難：日付データ"data"のうち，各国の最新データを元にデータフレームを作成

countries = df.columns # 列の国名コード
date_keys = pd.DataFrame(df['USA']['data']).columns # dataに含まれるcolumn名（アメリカより取得）

df2 = pd.DataFrame(columns = countries, index = date_keys) # index：感染情報，column：国名コードとなるdf2を作成する

# 各国ごとに最新の有効データを取得
for country in countries:
    # 各国のデータをDataFrameに変換
    df_country = pd.DataFrame(df[country]['data'])
    # 最新の有効なデータを保持する辞書を初期化
    latest_data = {}
    
    # 各列について最後の有効なデータを検索
    for column in df_country.columns:
        last_valid_index = df_country[column].last_valid_index()
        if last_valid_index is not None:
            latest_data[column] = df_country.loc[last_valid_index, column]
        else:
            latest_data[column] = np.nan
    
    # 新しいDataFrameにデータを格納
    df2[country] = pd.Series(latest_data)

df_all = pd.concat([df,df2]).T
df_all['index'] = df_all.index # foliumのために国名コードのindexもcolumnに追加する

# df_allの表示
df_all

In [ ]:
df_all.columns # columnの表示

In [ ]:
# 国形状のデータを取得する
url = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
world_geo = f"{url}/world-countries.json"

# foliumの表示の初期設定
m = folium.Map(location=[0, 0], zoom_start=2)

# foliumのオプション
folium.Choropleth(
    geo_data=world_geo, # 世界の国の形状を用いる
    name="choropleth",
    data=df_all, # OWIDのデータを使う
    columns=["index", "total_cases_per_million"], # dfのうちindex（国名コード），total_cases_per_millionのcolumnを使う
    key_on="feature.id", # world_geoのfeature.idと国名コードが一致するとき，その国を色塗り
    fill_color="YlOrRd", # 色の設定
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='black', # データがない国の色を設定
    legend_name="total_cases_per_million", # 凡例のラベル
).add_to(m)

folium.LayerControl().add_to(m)

m # 表示